# COMP0005 - GROUP COURSEWORK
# Experimental Evaluation of Search Data Structures and Algorithms

The cell below defines **AbstractSearchInterface**, an interface to support basic insert/search operations; you will need to implement this three times, to realise your three search data structures of choice among: (1) *2-3 Tree*, (2) *AVL Tree*, (3) *LLRB BST*; (4) *B-Tree*; and (5) *Scapegoat Tree*. <br><br>**Do NOT modify the next cell** - use the dedicated cells further below for your implementation instead. <br>

In [ ]:
# DO NOT MODIFY THIS CELL

from abc import ABC, abstractmethod  

class AbstractSearchInterface(ABC):
    '''
    Abstract class to support search/insert operations (plus underlying data structure)
    
    '''
        
    @abstractmethod
    def insertElement(self, element):     
        '''
        Insert an element in a search tree
            Parameters:
                    element: string to be inserted in the search tree (string)

            Returns:
                    "True" after successful insertion, "False" if element is already present (bool)
        '''
        
        pass 
    

    @abstractmethod
    def searchElement(self, element):
        '''
        Search for an element in a search tree
            Parameters:
                    element: string to be searched in the search tree (string)

            Returns:
                    "True" if element is found, "False" otherwise (bool)
        '''

        pass

Use the cell below to define any auxiliary data structure and python function you may need. Leave the implementation of the main API to the next code cells instead.

In [ ]:
# ADD AUXILIARY DATA STRUCTURE DEFINITIONS AND HELPER CODE HERE
class Node23:
    def __init__(self, is_leaf=True):
        self.keys = []
        self.children = []
        self.is_leaf = is_leaf

class NodeAVL:
    def __init__(self, key, parent=None):
        self.key = key
        self.left = None
        self.right = None
        self.parent = parent
        self.height = 1
    

Use the cell below to implement the requested API by means of **2-3 Tree** (if among your chosen data structure).

In [3]:
class TwoThreeTree(AbstractSearchInterface):
    def __init__(self):
        self.root = None

    def searchElement(self, key):
        if self.root is None:
            return False
        else:
            return self.search_recursively(self.root,key)

    def insertElement(self, key):
        if self.root is None:
            self.root = Node23(is_leaf=True)
            self.root.keys.append(key)
            return True
        success, promotion = self.insert_recursively(self.root, key)
        if not success:
            return False
        if promotion is not None: # deal with the root
            mid_key, new_left, new_right = promotion
            new_root = Node23(is_leaf=False)
            new_root.keys.append(mid_key)
            new_root.children = [new_left, new_right]
            self.root = new_root
        return True

    def insert_recursively(self, node, key):
        if key in node.keys:
            return (False, None)

        if node.is_leaf :
            self.insert_to_left(node, key)
            if len(node.keys) > 2:
                promotion = self.split_leaf(node)
                return (True, promotion)
            return (True, None)
        else :
            children_index = self.find_children_index(node,key)
            success, promotion = self.insert_recursively(node.children[children_index],key)
            if not success:
                return (False, None)
            if promotion is not None: #deal with the parent node in each lays
                mid_key,new_left,new_right = promotion
                self.merge(node,mid_key,new_left,new_right)
                if len(node.keys) > 2:
                    promotion_internal = self.split_internal(node)
                    return (True, promotion_internal)
            return (True, None)

    def split_leaf(self, node):
        mid_key = node.keys[1]
        new_left = Node23(is_leaf=True)
        new_left.keys.append(node.keys[0])
        new_left.children = []
        new_right = Node23(is_leaf=True)
        new_right.keys.append(node.keys[2])
        new_right.children = []
        return mid_key,new_left,new_right

    def split_internal(self, node):
        mid_key = node.keys.pop(1)
        new_left = Node23(is_leaf=False)
        new_left.keys.append(node.keys[0])
        new_left.children =node.children[0:2]
        new_right = Node23(is_leaf=False)
        new_right.keys.append(node.keys[1])
        new_right.children =node.children[2:]
        return mid_key,new_left,new_right

    def merge(self, node, mid_key, new_left, new_right):
        idx = self.find_children_index(node, mid_key)
        node.keys.insert(idx, mid_key)
        node.children[idx] =new_left
        node.children.insert(idx + 1, new_right)

    def insert_to_left(self, node, key):
        idx = 0
        while idx < len(node.keys) and key > node.keys[idx]:
            idx += 1
        node.keys.insert(idx, key)

    def find_children_index(self, node, key):
        idx = 0
        while idx < len(node.keys) and key > node.keys[idx]:
            idx += 1
        return idx

    def search_recursively(self, node, key):
         n = len(node.keys)
         for idx in range(n):
            if node.keys[idx] == key:
                return True
         if node.is_leaf :
            return False
         else:
            children_index = self.find_children_index(node, key)
            return self.search_recursively(node.children[children_index],key)

NameError: name 'AbstractSearchInterface' is not defined

Use the cell below to implement the requested API by means of **AVL Tree** (if among your chosen data structure).

In [ ]:
class AVLTree(AbstractSearchInterface):
    inserted = True

    def __init__(self):
        self.root = None

    def getHeight(self, node):
        return node.height if node else 0

    def getBalanceFactor(self, node):
        if not node:
            return 0
        return self.getHeight(node.left) - self.getHeight(node.right)

    def updateHeight(self, node):
        if node:
            node.height = 1 + max(self.getHeight(node.left), self.getHeight(node.right))

    def leftRotate(self, node):

        rightChild = node.right
        if not rightChild:
            return node

        T2 = rightChild.left

        rightChild.left = node
        node.right = T2

        rightChild.parent = node.parent
        node.parent = rightChild
        if T2:
            T2.parent = node

        if rightChild.parent is None:
            self.root = rightChild
        else:
            if rightChild.parent.left == node:
                rightChild.parent.left = rightChild
            else:
                rightChild.parent.right = rightChild

        self.updateHeight(node)
        self.updateHeight(rightChild)
        return rightChild

    def rightRotate(self, node):
        leftChild = node.left
        if not leftChild:
            return node

        T2 = leftChild.right

        leftChild.right = node
        node.left = T2

        leftChild.parent = node.parent
        node.parent = leftChild
        if T2:
            T2.parent = node

        if leftChild.parent is None:
            self.root = leftChild
        else:
            if leftChild.parent.left == node:
                leftChild.parent.left = leftChild
            else:
                leftChild.parent.right = leftChild

        self.updateHeight(node)
        self.updateHeight(leftChild)
        return leftChild

    def insert(self, key):
        self.root = self._insert(self.root, key, parent=None)

    def _insert(self, node, key, parent):
        global inserted
        if not node:
            return NodeAVL(key, parent)
        if key < node.key:
            node.left = self._insert(node.left, key, node)
        elif key > node.key:
            node.right = self._insert(node.right, key, node)
        else:
            # Duplicate keys are not allowed
            inserted = False
            return node

        self.updateHeight(node)
        balance = self.getBalanceFactor(node)

        # Left Left Case
        if balance > 1 and node.left and key < node.left.key:
            return self.rightRotate(node)
        # Right Right Case
        if balance < -1 and node.right and key > node.right.key:
            return self.leftRotate(node)
        # Left Right Case
        if balance > 1 and key > node.left.key:
            node.left = self.leftRotate(node.left)
            return self.rightRotate(node)
        # Right Left Case
        if balance < -1 and key < node.right.key:
            node.right = self.rightRotate(node.right)
            return self.leftRotate(node)

        return node

    def print_tree(self,node, level=0):
        if node is None:
            return
        print(f"Level {level}: {node.key}")
        self.print_tree(node.right, level + 1)
        self.print_tree(node.left, level + 1)
        
    def insertElement(self, element):
        global inserted 
        inserted = True
        # ADD YOUR CODE HERE
        self.insert(element)
        return inserted
    

    def searchElement(self, element):     
        found = False
        # ADD YOUR CODE HERE
        node = self.root
        while node:
            if node.key == element:
                found = True
            elif element < node.key:
                node = node.left
            else:
                node = node.right
        
        return found  

NameError: name 'AbstractSearchInterface' is not defined

Use the cell below to implement the requested API by means of **LLRB BST** (if among your chosen data structure).

In [ ]:
class LLRBBST(AbstractSearchInterface):
        
    def insertElement(self, element):
        inserted = False
        # ADD YOUR CODE HERE
      
        
        return inserted
    
    

    def searchElement(self, element):     
        found = False
        # ADD YOUR CODE HERE

        
        return found  

Use the cell below to implement the requested API by means of **B-Tree** (if among your chosen data structure).

In [ ]:
class BTree(AbstractSearchInterface):
        
    def insertElement(self, element):
        inserted = False
        # ADD YOUR CODE HERE
      
        
        return inserted
    
    

    def searchElement(self, element):     
        found = False
        # ADD YOUR CODE HERE

        
        return found

Use the cell below to implement the requested API by means of **Scapegoat Tree** (if among your chosen data structure).

In [ ]:
class ScapegoatTree(AbstractSearchInterface):
        
    def insertElement(self, element):
        inserted = False
        # ADD YOUR CODE HERE
      
        
        return inserted
    
    

    def searchElement(self, element):     
        found = False
        # ADD YOUR CODE HERE

        
        return found 

Use the cell below to implement the **synthetic data generator** needed by your experimental framework (be mindful of code readability and reusability).

In [ ]:
import string
import random

class TestDataGenerator():
    '''
    A class to represent a synthetic data generator.

    ...

    Attributes
    ----------
    
    [to be defined as part of the coursework]

    Methods
    -------
    
    [to be defined as part of the coursework]

    '''
    
    #ADD YOUR CODE HERE
    
    def __init__(self):
        pass
        
    def generate_random_string(self, length: int) -> str:
        return ''.join(random.choice(string.ascii_lowercase) for _ in range(length))
    
    def generate_dataset(self, n: int, str_length: int = 10) -> list[str]:
        """Generate a dataset of n unique random strings"""
        unique_strings = set()
        while len(unique_strings) < n:
            unique_strings.add(self.generate_random_string(str_length))
        return list(unique_strings)
    
    def load_dataset_from_file(self,file_path: str) -> list[str]:
        """Load a dataset from a file"""
        with open(file_path, 'r') as file:
            dataset = [line.strip() for line in file.readlines()]
        return dataset
    
    def generate_dataset_normal_distribution(self,n: int, str_length: int = 10) -> list[str]:
        """Generate a dataset of n unique random strings with a normal distribution"""
        mean = n // 2
        std_dev = n // 4
        dataset = []
        for _ in range(n):
            dataset.append(str(int(random.gauss(mean, std_dev)))) 
        return dataset
    
    def generate_dateset_with_duplicates(self,n: int, str_length: int = 10) -> list[str]:
        """Generate a dataset of n unique random strings with duplicates"""
        dataset = self.generate_dataset(n, str_length)
        duplicates = random.sample(dataset, n // 10)  # 10% duplicates
        dataset.extend(duplicates)
        return dataset
    
    def generate_sorted_dataset(self,n: int, str_length: int = 10, reverse = False) -> list[str]:
        """Generate a sorted dataset of n unique random strings"""
        """Represent edge cases for the dataset"""
        dataset = self.generate_dataset(n, str_length)
        dataset.sort(reverse=reverse)
        return dataset
        

Use the cell below to implement the requested **experimental framework** (be mindful of code readability and reusability).

In [ ]:
import timeit
import matplotlib

class ExperimentalFramework():
    '''
    A class to represent an experimental framework.

    ...

    Attributes
    ----------
    
    [to be defined as part of the coursework]

    Methods
    -------
    
    [to be defined as part of the coursework]

    '''
            
    #ADD YOUR CODE HERE
    
    def __init__(self):
        pass
        
    def benchmark_insertion(self, data_structure, dataset: list[str]) -> float:
        start_time = timeit.default_timer()
        for element in dataset:
            data_structure.insertElement(element)
        end_time = timeit.default_timer()
        return end_time - start_time

    def benchmark_search(self, data_structure, dataset: list[str]) -> tuple[float, float]:
        start_time = timeit.default_timer()
        for element in dataset:
            data_structure.searchElement(element)
        end_time = timeit.default_timer()
        existing_search_time = end_time - start_time
        
        # Search for non-existing elements
        non_existing_elements = [f"non_existing_{i}" for i in range(len(dataset))]
        start_time = timeit.default_timer()
        for element in non_existing_elements:
            data_structure.searchElement(element)
        end_time = timeit.default_timer()
        non_existing_search_time = end_time - start_time
        return existing_search_time, non_existing_search_time
    

Use the cell below to illustrate the python code you used to **fully evaluate** your three chosen search data structures and algortihms. The code below should illustrate, for example, how you made used of the **TestDataGenerator** class to generate test data of various size and properties; how you instatiated the **ExperimentalFramework** class to  evaluate each data structure using such data, collect information about their execution time, plot results, etc. Any results you illustrate in the companion PDF report should have been generated using the code below.

In [ ]:
# ADD YOUR TEST CODE HERE 

